# DIVALGO demonstration

This notebook contains a demostration of the DIVALGO tool

In [1]:
import streamlit as st
import importlib
import divalgo_class
importlib.reload(divalgo_class)
import divalgo_class as div
import os
import sklearn.linear_model as lm
import numpy as np
from PIL import Image
from sklearn import metrics

2022-06-08 12:06:04.965 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


## Actual data - dogs and wolfs

In [2]:
# Load data
dogs = sorted(os.listdir(os.path.join("..", "data", "dogs")))
wolves =  sorted(os.listdir(os.path.join("..", "data", "wolves")))


In [3]:
# Preprocessing
img_size = 50
dogs_images = []
wolves_images = [] 

for i in dogs:
    if os.path.isfile(os.path.join("..","data", "dogs", f"{i}")):
        img = Image.open(os.path.join("..","data", "dogs", f"{i}")).convert('L')            
        img = img.resize((img_size,img_size), Image.ANTIALIAS)
        img = np.asarray(img)/255.0
        dogs_images.append(img)    

for i in wolves:
    if os.path.isfile(os.path.join("..","data", "wolves", f"{i}")):
        img = Image.open(os.path.join("..","data", "wolves", f"{i}")).convert('L')
        img = img.resize((img_size,img_size), Image.ANTIALIAS)
        img = np.asarray(img)/255.0     
        wolves_images.append(img)   

In [4]:
# # Manual train-test split (to track filenames)
# X_train = np.asarray(dogs_images[0:800] + wolves_images[0:800])
# X_train = X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2])
# X_test = np.asarray(dogs_images[800:1000] + wolves_images[800:1000])
# X_test = X_test.reshape(X_test.shape[0], X_test.shape[1]*X_test.shape[2])
# y_train = np.asarray(["dog" for y in range(800)] + ["wolf" for y in range(800)])
# y_train = y_train.reshape(y_train.shape[0],1)
# y_test_ar = np.asarray(["dog" for y in range(200)] + ["wolf" for y in range(200)])
# y_test = y_test_ar.reshape(y_test_ar.shape[0],1)

# y_train, y_test = [k.T for k in [y_train, y_test]]
# filenames_test = [os.path.join("..", "data", "dogs", d) for d in dogs[800:1000]] + [os.path.join("..", "data", "wolves", w) for w in wolves[800:1000]]

In [5]:
# Manual train-test split (to track filenames)
X_train = np.asarray(dogs_images[0:100] + wolves_images[0:100])
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2])
X_test = np.asarray(dogs_images[800:825] + wolves_images[800:825])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1]*X_test.shape[2])
y_train = np.asarray(["dog" for y in range(100)] + ["wolf" for y in range(100)])
y_train = y_train.reshape(y_train.shape[0],1)
y_test_ar = np.asarray(["dog" for y in range(25)] + ["wolf" for y in range(25)])
y_test = y_test_ar.reshape(y_test_ar.shape[0],1)

y_train, y_test = [k.T for k in [y_train, y_test]]
filenames_test = [os.path.join("..", "data", "dogs", d) for d in dogs[800:825]] + [os.path.join("..", "data", "wolves", w) for w in wolves[800:825]]

In [6]:
# Train model
model = lm.LogisticRegression(penalty='none', tol=0.1, max_iter=500).fit(X_train, y_train[0])

In [7]:
y_pred = model.predict(X_test)
dog_wolf = div.Evaluate((X_test, y_test[0], y_pred, filenames_test), model)

In [8]:
#dog_wolf.explore_embeddings()

In [9]:
dog_wolf.open_visualization()

2022-06-08 12:06:36.194 INFO    numexpr.utils: NumExpr defaulting to 8 threads.



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8533
  Network URL: http://192.168.8.150:8533

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            


50it [00:04, 10.18it/s]


Training: 

Train - epoch - loss over time, learning curves (Frida)

Coefficient heatmap (Jakob)

Visualize images (correct/wrong predictions) - Stine 